In [159]:
import pandas as pd
import numpy as np
import re

In [160]:
# Load the CSV file
file_path = r"C:\Users\User\Desktop\old expedia\last_output_df_20_06_2024.csv"
OUTPUT_FILE_PATH = r"C:\Users\User\Desktop\expedia_clean_dfs\last_output_df_20_06_2024.csv"
df = pd.read_csv(file_path)

In [161]:
df.head()

,Snapshot,start_date,end_dateIndex,Hotel Name,TTT,LOS,Grade,Num of Reviews,Curr Price,Original Price,Percentage of discount,Distance from center,Type of room,Location grade,Is refundable,Late payment,Breakfast included,Option Member,end_date,Index
0,2024-06-20 13:47:04.556652,2024-06-21,NaN,"PUBLIC, an Ian Schrager hotel",1.0,1.0,8.28,2589,3339.0,0.0,-inf,0.0,New York,0.0,False,False,False,True,2024-06-22,0.0
1,2024-06-20 13:47:04.585605,2024-06-21,NaN,"Club Quarters Hotel, Grand Central",1.0,1.0,8.68,1950,179.0,199.0,10.050251,0.0,New York,0.0,True,False,False,True,2024-06-22,1.0
2,2024-06-20 13:47:04.603553,2024-06-21,NaN,Hotel Boutique at Grand Central,1.0,1.0,8.68,1052,179.0,199.0,10.050251,0.0,New York,0.0,True,False,False,True,2024-06-22,2.0
3,2024-06-20 13:47:04.617519,2024-06-21,NaN,Paramount Times Square,1.0,1.0,NaN,2560,159.0,0.0,-inf,0.0,Theater District,0.0,False,False,False,False,2024-06-22,3.0
4,2024-06-20 13:47:04.630486,2024-06-21,NaN,World Center Hotel,1.0,1.0,8.88,2020,197.0,219.0,10.045662,0.0,New York,0.0,True,False,False,True,2024-06-22,4.0


In [162]:
df = df.drop(["Location grade", "Distance from center"], axis=1)

In [163]:
# change columns names
df = df.rename(columns={'Hotel Name': "name", "Grade": "score", "Num of Reviews": "reviews", "Curr Price": "price_per_night", "Original Price": "original_price", "Percentage of discount": "discount_perc"})
df = df.rename(columns={"Is refundable": "Free_cancellation", "Late payment": "No_payment", "Breakfast included": "Breakfast", "rating-star": "star_rating"})

In [164]:
# Add data from booking df on same hotels
booking_df = pd.read_csv(r"C:\Users\User\Downloads\20_06_hotels_data_cleaned (1).csv")
booking_df.columns


Index(['name', 'location', 'score', 'reviews', 'room_type', 'bed_type',
       'price', 'availability', 'url', 'star_rating', 'location_rating',
       'Snapshot', 'Free_cancellation', 'No_prepayment', 'Limited_rooms',
       'TTT', 'Breakfast', 'index', 'neighborhood', 'km_from_center', 'LOS',
       'price_per_night', 'room_rate', 'neighborhood_category'],
      dtype='object')

In [165]:
def copy_data_from_booking_hotels(booking_df, expedia_df):
    unique_booking_hotels_data = booking_df.drop_duplicates(subset="name")
    expedia_df = expedia_df.merge(unique_booking_hotels_data[["name", "star_rating", "location_rating", "neighborhood", "km_from_center", "neighborhood_category"]], on="name", how='left')
    return expedia_df

updated_df = copy_data_from_booking_hotels(booking_df, df)
updated_df.columns

Index(['Snapshot', 'start_date', 'end_dateIndex', 'name', 'TTT', 'LOS',
       'score', 'reviews', 'price_per_night', 'original_price',
       'discount_perc', 'Type of room', 'Free_cancellation', 'No_payment',
       'Breakfast', 'Option Member', 'end_date', 'Index', 'star_rating',
       'location_rating', 'neighborhood', 'km_from_center',
       'neighborhood_category'],
      dtype='object')

In [166]:
def clean_data(df):
    # Remove non-numeric characters and convert to float for 'score'
    df['score'] = df['score'].astype(str).str.extract(r'(\d+\.\d+)')[0].astype(float)
    
    # Remove non-numeric characters for 'reviews' and handle empty strings
    df['reviews'] = df['reviews'].astype(str).str.replace(r'\D', '', regex=True)
    df['reviews'] = pd.to_numeric(df['reviews'], errors='coerce').fillna(0).astype(int)
    
    # Remove non-numeric characters for 'price_per_night' and handle empty strings
    df['price_per_night'] = df['price_per_night'].astype(str).str.replace(r'[^\d.]', '', regex=True)
    df['price_per_night'] = pd.to_numeric(df['price_per_night'], errors='coerce').fillna(0.0)

    # df['price_per_night'] = df['price_per_night'] / 100
    
    # Remove non-numeric characters for 'original_price' and handle empty strings
    df['original_price'] = df['original_price'].astype(str).str.replace(r'[^\d.]', '', regex=True)
    df['original_price'] = pd.to_numeric(df['original_price'], errors='coerce').fillna(0.0)
    
    # Convert 'Breakfast included' to string and update column to True/False
    df['Breakfast'] = df['Breakfast'].astype(str).str.contains('breakfast', case=False, na=False)
    
    return df

In [167]:
cleaned_df = clean_data(updated_df)

In [168]:
# Display the cleaned DataFrame
cleaned_df

,Snapshot,start_date,end_dateIndex,name,TTT,LOS,score,reviews,price_per_night,original_price,...,No_payment,Breakfast,Option Member,end_date,Index,star_rating,location_rating,neighborhood,km_from_center,neighborhood_category
0,2024-06-20 13:47:04.556652,2024-06-21,NaN,"PUBLIC, an Ian Schrager hotel",1.0,1.0,8.28,2589,3339.0,0.0,...,False,False,True,2024-06-22,0.0,NaN,NaN,NaN,NaN,NaN
1,2024-06-20 13:47:04.585605,2024-06-21,NaN,"Club Quarters Hotel, Grand Central",1.0,1.0,8.68,1950,179.0,199.0,...,False,False,True,2024-06-22,1.0,NaN,NaN,NaN,NaN,NaN
2,2024-06-20 13:47:04.603553,2024-06-21,NaN,Hotel Boutique at Grand Central,1.0,1.0,8.68,1052,179.0,199.0,...,False,False,True,2024-06-22,2.0,4.0,NaN,Midtown East,1.8,Midtown Manhattan
3,2024-06-20 13:47:04.617519,2024-06-21,NaN,Paramount Times Square,1.0,1.0,NaN,2560,159.0,0.0,...,False,False,False,2024-06-22,3.0,NaN,NaN,NaN,NaN,NaN
4,2024-06-20 13:47:04.630486,2024-06-21,NaN,World Center Hotel,1.0,1.0,8.88,2020,197.0,219.0,...,False,False,True,2024-06-22,4.0,4.0,8.5,Manhattan,7.0,Other Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37995,2024-06-20 15:05:39.134807,2024-07-20,NaN,Untitled at 3 Freeman Alley,30.0,5.0,9.29,244,262.0,264.0,...,False,False,True,2024-07-25,246.0,NaN,NaN,NaN,NaN,NaN
37996,2024-06-20 15:05:39.144780,2024-07-20,NaN,TownePlace Suites by Marriott New York Manhattan,30.0,5.0,8.88,1000,296.0,0.0,...,False,False,False,2024-07-25,247.0,NaN,NaN,NaN,NaN,NaN
37997,2024-06-20 15:05:39.154784,2024-07-20,NaN,Sonder Duane Street,30.0,5.0,8.48,735,210.0,259.0,...,False,False,False,2024-07-25,248.0,4.0,NaN,Tribeca,6.2,Downtown Manhattan
37998,2024-06-20 15:05:39.166754,2024-07-20,NaN,Arlo NoMad,30.0,5.0,8.48,1318,319.0,335.0,...,False,False,True,2024-07-25,249.0,4.0,NaN,NoMad,2.4,Downtown Manhattan


In [169]:
cleaned_df.to_csv(OUTPUT_FILE_PATH, index=False)